## 1. Python을 통한 DB Connection

### 1-1 Postgresql 라이브러리(psycopg2)를 활용한 Connection

##### 1) Connection

In [ ]:
import psycopg2

# database connection 생성
db = psycopg2.connect(
    host='localhost',
    dbname='postgres',
    user='hyunsoo',
    password='150808',
    port=5432
    )

# 커서 생성 > 특정 SQL 문장을 처리한 결과를 담고 있는 영역을 가리키는 일종의 포인터 / 쿼리문에 의해서 반환되는 결과값들을 저장하는 메모리공간
cursor=db.cursor()

##### 2) CRUD Execution

In [ ]:
create_query = "CREATE TABLE lecture (id SERIAL PRIMARY KEY, name VARCHAR(32), year INT, gender VARCHAR(10), count INT);"

insert_query = "INSERT INTO lecture VALUES(1, 'Tom', 2023, 'M', 1100);"

update_query = """
            UPDATE lecture 
            SET id = 1,
                name = 'HYUNSOO',
                year = '1991',
                gender = 'M',
                count = '1000'
            WHERE id = 1;
            """
delete_query = "DELETE FROM lecture WHERE id = 1;"

drop_query = "DROP TABLE lecture"

In [ ]:
# CREATE TABLE

cursor.execute(create_query)
db.commit()

In [ ]:
# INSERT ROW

cursor.execute(insert_query)
db.commit()

cursor.execute("SELECT * FROM lecture")
cursor.fetchall()

In [ ]:
# UPDATE ROW

cursor.execute(update_query)
db.commit()

cursor.execute("SELECT * FROM lecture")
cursor.fetchall()

In [ ]:
# DELETE ROW

cursor.execute(delete_query)
db.commit()

cursor.execute("SELECT * FROM lecture")
cursor.fetchall()

In [ ]:
# DROP TABLE

cursor.execute(drop_query)
db.commit()

#### 3) Connection Pool

<img src="https://velog.velcdn.com/images/newnew_daddy/post/f0569aa7-1aad-466e-a24f-5e3b5f248a72/image.png" width="30%">

- 일정량의 Connection 객체를 미리 만들어서 pool에 저장
- 클라이언트 요청이 오면 Connection 객체를 빌려주고 해당 객체의 임무가 완료되면 다시 Connection 객체를 반납 받아 pool에 저장
- 큰 커넥션 풀은 메모리 소모가 큰 대신 대기 시간이 적어지고, 작은 커넥션 풀은 메모리 소모가 작은 대신 대기 시간이 길어진다. ([적정 Connection 수 공식](https://github.com/brettwooldridge/HikariCP/wiki/About-Pool-Sizing#the-formula))
- 자원을 사용하면 반드시 반납을 해줘야하는데 이를 위해 DB 연결시 파이썬 `with문`을 사용

In [ ]:
## Connection 종료

cursor.close()

### 1-2. ORM 라이브러리(sqlalchemy)를 활용한 Connection

#### ORM(Object Relational Mapping) 이란?

<img src="https://velog.velcdn.com/images/newnew_daddy/post/ca5652a2-0686-462b-bed4-4483815708e5/image.png" width="40%">

- Python 객체와 관계형 DB의 data를 매핑해주는 것
- 장점
    - Query가 아닌 코드로 데이터를 조작할 수 있어 개발자가 프로그래밍에 더 집중할 수 있도록 도와준다.
    - 재사용 및 유지보수의 편리성이 증가한다.
    - DBMS에 대한 종속성이 줄어든다.

- 단점
  - 완벽한 ORM으로만 서비스를 구현하기 어렵다.
  - 프로시저가 많은 시스템에선 ORM의 객체 지향적인 장점을 활용하기 어렵다.

##### 1) Connection 및 Session

In [ ]:
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

engine_name = 'postgresql'
user_id = 'hyunsoo'
user_pw = '150808'
host = 'localhost'
port = '5432'
database = 'postgres'

db = create_engine(f'{engine_name}://{user_id}:{user_pw}@{host}:{port}/{database}')

cursor = db.connect()
Session = sessionmaker(db)
session = Session()


In [ ]:
insert_query = "INSERT INTO lecture VALUES(3, 'Mom', 2023, 'M', 1100);"

cursor.execute(text(insert_query))
cursor.commit()

In [ ]:
cursor.execute(text("SELECT * FROM lecture"))

In [ ]:
cursor.close()

In [ ]:
import pandas as pd

pd.read_sql("select * from lecture", db)

##### 2) 매핑 테이블 생성

In [ ]:
from sqlalchemy import Column, INTEGER, VARCHAR
from sqlalchemy.orm import declarative_base

Base = declarative_base()

class lecture(Base):
    __tablename__ = 'lecture'
    
    id = Column('id', INTEGER, primary_key=True, autoincrement=True)
    name = Column('name', VARCHAR(10), nullable=False)
    year = Column('year', INTEGER, nullable=False)
    gender = Column('gender', VARCHAR(10), nullable=False)
    count = Column('count', INTEGER, nullable=False)

# Create
Base.metadata.create_all(db)

# Drop
# Sample.__table__.drop(db)

##### 3) ORM을 활용한 CRUD

In [ ]:
# 1) SELECT ONE

res = session.query(lecture).first()

(res.id, res.name, res.year, res.gender, res.count)

In [ ]:
# 1) SELECT ALL

res = session.query(lecture).all()

for i in res:
    print(i.id, i.name, i.year, i.gender, i.count)

In [ ]:
# 2) INSERT

data1 = lecture(id=3, name='hyunsoo', year=1990, gender='M', count=1234)
session.add(data1)
session.commit()

In [ ]:
# 3) UPDATE

session.query(lecture).filter(lecture.name == 'hyunsoo').update({'gender':'F', 'count':1500})
session.commit()

In [ ]:
# 4) DELETE

session.query(lecture).filter(lecture.id == 2).delete()
session.commit()

##### 4) OMR Core 방법을 활용한 CRUD - 참고용
  > https://velog.io/@newnew_daddy/python-orm

In [ ]:
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData
from sqlalchemy.orm import sessionmaker

db = create_engine('postgresql://[USER ID]:[USER PW]@[IP]:[PORT]/[DATABASE]')

# 세션 : 
Session = sessionmaker(db)
session = Session()

meta = MetaData()

# 테이블 스키마에 맞게 Column 생성 후 테이블 선언
core_table = Table(
    'first', meta,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('name', String),
    Column('year', Integer),
    Column('gender', String),
    Column('count', Integer)
)

meta.create_all(db)

In [ ]:
from sqlalchemy import insert, update, delete

# SELECT ALL
res = core_table.select()
result = session.execute(res)

for row in result:
    print(row)
    

# INSERT 1건
stmt = insert(core_table).values(name="hahaha", year=2023, gender="M", count=654)

with db.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()


# INSERT 다수
stmt = insert(core_table)
data_list = [
    {"name": "tom", "year": 2023, "gender": "M", "count": 12},
    {"name": "ann", "year": 2000, "gender": "F", "count": 32}
   ]

with db.connect() as conn:
    result = conn.execute(stmt, data_list)
    conn.commit()
    
# UPDATE
stmt = update(core_table).where(core_table.c.name == 'ann').values(year=1990)

with db.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()
    
# DELETE
stmt = delete(core_table).where(core_table.c.name == 'ann')

with db.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

### 1-3. Python을 활용한 Bulk Insert
1. For문을 활용한 Insert
2. `to_sql()` 함수를 활용한 Insert
3. `execute_values()` 함수를 활용한 Insert

> https://velog.io/@newnew_daddy/PYTHON05

##### 1) Pandas for loop

In [ ]:
import psycopg2

# database connection 생성
db = psycopg2.connect(
    host='localhost',
    dbname='postgres',
    user='hyunsoo',
    password='150808',
    port=5432
    )

# 커서 생성 > 특정 SQL 문장을 처리한 결과를 담고 있는 영역을 가리키는 일종의 포인터 / 쿼리문에 의해서 반환되는 결과값들을 저장하는 메모리공간
cursor=db.cursor()

In [ ]:
import pandas as pd

# DATA IMPORT
df = pd.read_csv('../dataset/data-01/names.csv')

# Index Location
# df.loc[1, :]
df.iloc[1]

In [ ]:
# BULK INSERT -> tuple 형태로 감싸서 Insert!

for data in range(len(df)):
    datas = tuple(df.iloc[data])
    query = f"INSERT INTO lecture VALUES{datas};"
    cursor.execute(query)
    
db.commit()

In [ ]:
# CHECK

cursor.execute("SELECT * FROM first LIMIT 10")
cursor.fetchall()

##### 2) Pandas - [to_sql()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html) 메소드
- dataframe.to_sql( 테이블 이름, sqlalchemy_connector, 옵션 )

In [ ]:
from sqlalchemy import create_engine

engine = 'postgresql'
user = 'hyunsoo'
passwd = '910506'
host = 'localhost'
port = '5432'
database = 'postgres'

db = create_engine(f'{engine}://{user}:{passwd}@{host}:{port}/{database}')

In [ ]:
import pandas as pd

# DATA IMPORT
df = pd.read_csv('../dataset/data-01/names.csv')
df.head()
# 50개만 선택
df = df.head(50)

In [ ]:
df.dtypes

In [ ]:
# BULK INSERT
df.to_sql(name="lecture", con=db, if_exists="replace")

##### 3) psycopg2 라이브러리의 [execute_values()](https://www.psycopg.org/docs/extras.html#psycopg2.extras.execute_values) 메소드
- 사용 예시 : https://int-i.github.io/python/2022-02-27/python-postgres-psycopg2-bulk-insert/

In [ ]:
import psycopg2

# database connection 생성
db = psycopg2.connect(
    host='localhost',
    dbname='postgres',
    user='hyunsoo',
    password='150808',
    port=5432
    )

# 커서 생성 > 특정 SQL 문장을 처리한 결과를 담고 있는 영역을 가리키는 일종의 포인터 / 쿼리문에 의해서 반환되는 결과값들을 저장하는 메모리공간
cursor=db.cursor()

In [ ]:
import pandas as pd

# DATA IMPORT
df = pd.read_csv('../dataset/data-01/names.csv')
df['id'] = df['id'].astype(float)
df['year'] = df['year'].astype(float)
df['count'] = df['count'].astype(float)

# 상위 50개 row만 tuple 형태로 List에 삽입
insert_list = list()

for i in range(5,50):
    insert_list.append(tuple(df.iloc[i]))
    
# df.apply(tuple, axis=1)[:50].tolist()

In [ ]:
df.dtypes

In [ ]:
import numpy as np
from psycopg2.extensions import register_adapter, AsIs
register_adapter(np.int64, AsIs)

In [ ]:
from psycopg2.extras import execute_values

sql = f"INSERT INTO lecture VALUES %s;"
execute_values(cursor, sql, insert_list)

db.commit()

## 2. dotenv 라이브러리를 활용한 민감 정보 관리

- 환경 변수에 대한 관리를 효과적이고 안전하게 할 수 있도록 도와주는 python 라이브러리
- DB정보, 비밀번호, API KEY 등 외부에 공유되거나 Git에 올라가면 안되는 값들을 하드코딩 하지 않고 사용이 가능.

    ```
    pip install python-dotenv
    ```

    > https://velog.io/@newnew_daddy/python-dotenv

##### 1) 기본 기능 사용

In [ ]:
import dotenv

# .env 경로 찾기
env_path = dotenv.find_dotenv()
print(env_path)
# .env 파일 load
dotenv.load_dotenv(env_path)

# .env 파일 변수값 출력
dotenv.dotenv_values(env_path)

# .env 파일 변수값 변경
# dotenv.set_key(env_path, '변경할 KEY', '변경할 VALUE')

In [ ]:
engine_name = 'postgresql'
user_id = 'learner01'
user_pw = '20231127'
host = 'localhost'
port = '5432'
database = 'postgres'

##### 2) .env 파일 작성

In [ ]:
POSTGRES_ENGINE='engine_name'
POSTGRES_HOST='localhost'
POSTGRES_DB_1='postgres'
POSTGRES_USER='learner01'
POSTGRES_PASSWORD='20231127'
POSTGRES_PORT='5432'

##### 3) 값들 Load해오기

In [ ]:
import os

engine = os.environ.get("POSTGRES_ENGINE")
host = os.environ.get("POSTGRES_HOST")
database = os.environ.get("POSTGRES_DB_1")
user = os.environ.get("POSTGRES_USER")
password = os.environ.get("POSTGRES_PASSWORD")
port = os.environ.get("POSTGRES_PORT")

(engine, host, database, user, password, port)

##### 4) DB Connection Test

In [ ]:
import psycopg2

db = psycopg2.connect(
    host = host,
    dbname = database,
    user = user,
    password = password,
    port = port
    )

cursor=db.cursor()

## 3. Python Class를 사용한 DB 접속 관리

##### 0) self에 대한 이해

In [ ]:
class TempClass:
    def __init__(self, name: str, age: int, count: int):
        self.name = name
        self.age = age
        self.count = count
        self.addage()
        
    def printall(self):
        print(self.name)
        print(self.age)
        print(self.count)
        
    def printself(self):
        return self
    
    def addage(self):
        self.new_age = self.age+10
        
a1 = TempClass("hyunsoo",2,3)
s1 = a1.printself()

In [ ]:
# 객체 인자들 확인

s1.__dict__

In [ ]:
[i for i in dir(s1) if '__' not in i]

In [ ]:
from easydict import EasyDict

self = EasyDict(dict())
self["name"] = "hyunsoo"
self["age"] = 2
self["count"] = 3

s2 = self

s2.age

##### 1) Class 작성

In [ ]:
class DBconnector:
    def __init__(self, host, database, user, password, port):
        self.host = host
        self.database = database
        self.user = user
        self.password = password
        self.port = port

    def postgres_connect(self):
        self.conn = psycopg2.connect(host=self.host,\
                                    dbname=self.database,\
                                    user=self.user, \
                                    password=self.password, \
                                    port=self.port)
        
        return self
    

In [ ]:
(host, database, user, password, port)

In [ ]:
db_object = DBconnector(host, database, user, password, port)
db_conn = db_object.postgres_connect().conn

cursor=db_conn.cursor()

In [ ]:
cursor.execute("SELECT * FROM lecture LIMIT 5")
cursor.fetchall()

##### 2) with문 적용
Bad Case

<img src="https://velog.velcdn.com/images/newnew_daddy/post/df312bda-0b22-4476-8a03-505f1d3cf5b4/image.png" width="20%">

Good Case

<img src="https://velog.velcdn.com/images/newnew_daddy/post/707d46b0-8a0b-4862-ad95-285dc04ddc29/image.png" width="20%">

- Python에서 파일 또는 리소스 관리를 더 효과적으로 처리하기 위한 블록 구조
- 리소스를 열고 사용한 후 자동으로 닫아주기 때문에 닫아주는 코드를 작성할 필요가 없습니다. 
- 주로 파일 입출력, 데이터베이스 연결, 네트워크 연결 등 리소스 관리에 사용됩니다.

    ```
    with문을 사용할 때는 with 키워드 다음에 리소스를 관리하는 객체를 생성하는 표현식을 사용하며, 
    이 객체는 __enter__와 __exit__ 메소드를 구현해야 합니다. 
    with 블록 내에서 리소스를 사용하고 블록을 벗어나면 __exit__ 메소드가 호출되어 리소스를 정리합니다.

    -ChatGPT-
    ```

In [ ]:
file = open('temp.txt', 'a')
file.write("1")
file.write("2")

In [ ]:
file.write("sadfsadf")
file.write("234")

In [ ]:
file.close()

In [ ]:
db_object = DBconnector(host, database, user, password, port)

with db_object as connected:
    db_conn = connected.conn
    cursor=db_conn.cursor()

    cursor.execute("SELECT * FROM lecture LIMIT 5")
    print(cursor.fetchall())

##### 3) enter, exit 적용

> https://docs.python.org/ko/3/reference/datamodel.html#object.__enter__

In [ ]:
class ContextManager:
    def __init__(self, age):
        print("Class 진입")
        self.age = age
        
    def __enter__(self):
        print("enter 시작")
        
    def __exit__(self, exc_type, exc_value, traceback):
        print("exit 종료")
        
    def haha(self):
        print(self.age)
        return self.__dict__

In [ ]:
context = ContextManager(10)
    
with context:
    print(context.haha())
    

In [ ]:
class DBconnector:
    def __init__(self, host, database, user, password, port):
        self.host = host
        self.database = database
        self.user = user
        self.password = password
        self.port = port
        self.connect = self.postgres_connect()

    def __enter__(self):
        print("접속")
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.conn.close()
        print("종료")

    def postgres_connect(self):
        self.conn = psycopg2.connect(host=self.host,\
                                    dbname=self.database,\
                                    user=self.user, \
                                    password=self.password, \
                                    port=self.port)
        
        return self
    

In [ ]:
db_object = DBconnector(host, database, user, password, port)
db_conn = db_object.postgres_connect().conn

cursor=db_conn.cursor()

In [ ]:
db_object = DBconnector(host, database, user, password, port)

with db_object as connected:
    db_conn = connected.conn
    cursor=db_conn.cursor()

    cursor.execute("SELECT * FROM lecture LIMIT 5")
    print(cursor.fetchall())



##### 4) connection parameter 합치기

In [ ]:
class DBconnector:
    def __init__(self, host, database, user, password, port):
        self.conn_params = dict(
            host = host,
            dbname = database,
            user = user,
            password = password,
            port=  port
        )
        self.host = host
        self.database = database
        self.user = user
        self.password = password
        self.port = port
        # self.connect = self.postgres_connect()

    def __enter__(self):
        print("접속")
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.conn.close()
        print("종료")

    def postgres_connect(self):
        self.conn = psycopg2.connect(host=self.host,\
                                    dbname=self.database,\
                                    user=self.user, \
                                    password=self.password, \
                                    port=self.port)
        
        return self
    

##### 5) Asterisk(*)에 대한 이해

In [ ]:
def func(one, two):
    print("one = ", one)
    print("two = ", two)

func(1,2)

In [ ]:
# *args -> 이름이 정해지지 않은 변수 여러개

def func(*args):
    print("args = ", args)
    print(sum(args))
    
func(1,2,3)

In [ ]:
# **kwargs -> 이름이 정해져있는 변수 여러개(key-value 조합)

def func(**kwargs):
    print("kwargs = ", kwargs)
    
func(one=1)

In [ ]:
# 개별 변수

test = DBconnector(1,2,3,4,5)

test.conn_params

In [ ]:
# List

args = [1,2,3,4,5]

list_params = DBconnector(*args)

list_params.conn_params

In [ ]:
# Dictionary

kwargs = dict(
            host = 1,
            database = 2,
            user = 3,
            password = 4,
            port=  5
        )

dict_params = DBconnector(**kwargs)

dict_params.conn_params

##### 6) DBconnector에 asterisk 적용

In [ ]:
class DBconnector:
    def __init__(self, host, database, user, password, port):
        self.conn_params = dict(
            host = host,
            dbname = database,
            user = user,
            password = password,
            port=  port
        )
        self.connect = self.postgres_connect()

    def __enter__(self):
        print("접속")
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.conn.close()
        print("종료")

    def postgres_connect(self):
        self.conn = psycopg2.connect(**self.conn_params)
        
        return self
    

In [ ]:
db_object = DBconnector(host, database, user, password, port)

with db_object as connected:
    db_conn = connected.conn
    cursor=db_conn.cursor()

    cursor.execute("SELECT * FROM lecture LIMIT 5")
    print(cursor.fetchall())

##### 7) .env 적용한 connection

In [ ]:
import dotenv, psycopg2

env_path = dotenv.find_dotenv()
dotenv.load_dotenv(env_path)

import os

engine = os.environ.get("POSTGRES_ENGINE")
host = os.environ.get("POSTGRES_HOST")
database = os.environ.get("POSTGRES_DB_1")
user = os.environ.get("POSTGRES_USER")
password = os.environ.get("POSTGRES_PASSWORD")
port = os.environ.get("POSTGRES_PORT")

In [ ]:
DB_SETTINGS = {
    "POSTGRES" : {
        # 'engine' : os.environ.get("POSTGRES_ENGINE"),
        'host' : os.environ.get("POSTGRES_HOST"),
        'database' : os.environ.get("POSTGRES_DB_1"),
        'user' : os.environ.get("POSTGRES_USER"),
        'password' : os.environ.get("POSTGRES_PASSWORD"),
        'port' : os.environ.get("POSTGRES_PORT")
    },
}

DB_SETTINGS['POSTGRES']

In [ ]:
db_object = DBconnector(**DB_SETTINGS['POSTGRES'])

# with db_object as connected:
#     db_conn = connected.conn
#     cursor=db_conn.cursor()

#     cursor.execute("SELECT * FROM lecture LIMIT 5")
#     print(cursor.fetchall())

##### 8) 다른 이름의 Database 생성 및 접속

In [ ]:
DB_SETTINGS = {
    "POSTGRES" : {
        'host' : os.environ.get("POSTGRES_HOST"),
        'database' : os.environ.get("POSTGRES_DB_1"),
        'user' : os.environ.get("POSTGRES_USER"),
        'password' : os.environ.get("POSTGRES_PASSWORD"),
        'port' : os.environ.get("POSTGRES_PORT")
    },
    "KDT" : {
        'host' : os.environ.get("POSTGRES_HOST"),
        'database' : os.environ.get("POSTGRES_DB_2"),
        'user' : os.environ.get("POSTGRES_USER"),
        'password' : os.environ.get("POSTGRES_PASSWORD"),
        'port' : os.environ.get("POSTGRES_PORT")
    }
}

DB_SETTINGS['KDT']

In [ ]:
db_object = DBconnector(**DB_SETTINGS['KDT'])

with db_object as connected:
    db_conn = connected.conn
    cursor=db_conn.cursor()

    cursor.execute("SELECT table_name FROM INFORMATION_SCHEMA.TABLES")
    print(cursor.fetchall())